In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import dendrogram, linkage

In [ ]:
df = pd.read_csv('/content/customer_features_scaled.csv')
df.head(10)

,CustomerID,Sum_Quantity,Mean_UnitPrice,Mean_TotalPrice,Sum_TotalPrice,Count_Invoice,Count_Stock,Mean_InvoiceCountPerStock,Mean_StockCountPerInvoice,Mean_UnitPriceMeanPerInvoice,Mean_QuantitySumPerInvoice,Mean_TotalPriceMeanPerInvoice,Mean_TotalPriceSumPerInvoice,Mean_UnitPriceMeanPerStock,Mean_QuantitySumPerStock,Mean_TotalPriceMeanPerStock,Mean_TotalPriceSumPerStock
0,000nan,4.984467,1.506343,-0.268408,4.595722,2.432481,4.773105,2.067962,2.330682,3.094080,0.761335,0.914696,2.100059,2.332104,1.979686,-0.076864,2.392380
1,012346,3.789233,-2.120154,3.800623,3.161700,-1.158633,-2.296006,-1.015864,-2.307499,-2.100457,8.847780,3.719362,7.377665,-2.141668,3.384916,3.763263,3.337067
2,012747,0.879252,0.879583,1.007144,1.397472,1.532615,0.156943,1.957285,-0.614124,0.749921,-0.306106,0.973710,0.485241,1.078802,1.041469,0.667948,1.526344
3,012748,3.038005,-0.106556,-1.068898,2.706127,2.352800,3.983682,1.982065,0.343236,1.872665,-0.238979,-0.080428,-0.710410,-0.049208,0.348309,-0.892187,-0.000881
4,012749,0.982814,1.125208,0.282588,1.379978,0.890138,1.397000,0.537992,1.095989,1.056342,0.756329,0.259518,1.527988,1.056653,-0.159892,0.290885,0.329836
5,012820,0.466642,-0.817031,-0.019920,0.296923,0.656870,0.395429,-0.412569,-0.123289,-0.824860,0.189404,-0.105017,-0.181951,-0.935154,0.238579,0.003600,-0.128951
6,012821,-1.233887,-0.235351,-0.060898,-1.725684,-1.158633,-1.361354,-1.015864,-1.051000,-0.261189,-0.848867,-0.151699,-1.489691,-0.233985,0.110322,-0.025617,-0.255883
7,012822,0.268811,-0.078187,0.286568,0.302366,-0.198605,0.135925,-0.081936,0.397982,-0.164225,0.676729,0.348066,0.785080,-0.141162,0.261716,0.260265,0.220589
8,012823,-0.366422,1.825216,2.528634,0.774998,0.890138,-2.296006,2.064151,-2.307499,1.669397,-1.283030,2.429015,0.374043,1.836621,2.299290,2.554670,2.796227
9,012824,-0.360106,-0.204132,-0.026684,-0.409951,-1.158633,-0.284543,-1.015864,0.500258,-0.231237,0.478007,-0.117808,0.540661,-0.202481,-0.149025,0.009031,-0.221666


In [ ]:
X = df.drop(columns=['CustomerID'])

Quy tắc khi chọn MinPts
| Quy tắc                       | Khi dùng      |
| ----------------------------- | ------------- |
| MinPts ≥ dim + 1              | cơ bản        |
| MinPts = 2 × dim              | phổ biến nhất |
| MinPts = 4 × dim              | dữ liệu nhiễu |
| MinPts = max(5, 2×dim)        | an toàn nhất  |
| Tăng MinPts theo size dataset | dữ liệu lớn   |


Quy tắc chọn eps:

1️⃣ Dùng K-distance plot để tìm elbow → chuẩn nhất

2️⃣ eps phải phản ánh mật độ thực của dữ liệu

3️⃣ eps phải áp dụng cho dữ liệu đã scale

4️⃣ MinPts tăng → eps tăng tương ứng

5️⃣ noise khoảng 10-30% là đẹp

6️⃣ eps không dựa cảm tính, phải dựa trên đồ thị

7️⃣ eps ≈ median(k-distance) (phụ trợ)

8️⃣ eps phụ thuộc vào số chiều

9️⃣ eps không quá nhỏ cũng không quá lớn

🔟 eps cuối cùng phải kiểm tra lại bằng trực quan (2D/3D)

In [ ]:
def select_best_eps_around_elbow(
    X,
    min_samples,
    eps_elbow,
    factors=None,
    min_cluster_points=10
):
    if eps_elbow is None:
        return None, None, []

    if factors is None:
        factors = np.linspace(0.8, 1.2, 9)

    best_row = None
    all_rows = []

    for f in factors:
        eps = float(eps_elbow * f)
        labels, n_clusters, noise_rate, sil, dbi = run_dbscan_with_metrics(
            X, eps, min_samples, min_cluster_points=min_cluster_points
        )

        row = {
            "eps": eps,
            "factor": float(f),
            "n_clusters": n_clusters,
            "noise_rate": float(noise_rate),
            "silhouette": sil,
            "dbi": dbi
        }
        all_rows.append(row)

        # chỉ xét cấu hình hợp lệ
        if sil is None or dbi is None or n_clusters < 2:
            continue

        if best_row is None:
            best_row = row
        else:
            # ưu tiên Sil cao, rồi DBI thấp, rồi noise thấp
            if (
                (row["silhouette"] > best_row["silhouette"]) or
                (row["silhouette"] == best_row["silhouette"] and row["dbi"] < best_row["dbi"]) or
                (row["silhouette"] == best_row["silhouette"] and row["dbi"] == best_row["dbi"]
                 and row["noise_rate"] < best_row["noise_rate"])
            ):
                best_row = row

    best_eps = None if best_row is None else best_row["eps"]
    return best_eps, best_row, all_rows


In [ ]:
def compute_eps_elbow(X, min_samples):
    """
    Tính eps tối ưu bằng k-distance elbow
    """
    n_samples = X.shape[0]
    k = max(min_samples - 1, 1)
    n_neighbors = min(k + 1, n_samples)

    if n_neighbors < 2:
        return None

    nbrs = NearestNeighbors(n_neighbors=n_neighbors).fit(X)
    distances, _ = nbrs.kneighbors(X)

    k_index = min(k, distances.shape[1] - 1)
    k_distances = np.sort(distances[:, k_index])

    # Elbow hình học
    n_points = len(k_distances)
    all_points = np.vstack((np.arange(n_points), k_distances)).T

    first_point = all_points[0]
    last_point = all_points[-1]
    line_vec = last_point - first_point
    norm = np.linalg.norm(line_vec)

    if norm == 0:
        return float(k_distances[-1])

    line_vec_norm = line_vec / norm
    vec_from_first = all_points - first_point
    scalar_proj = np.dot(vec_from_first, line_vec_norm)
    proj = np.outer(scalar_proj, line_vec_norm)
    vec_to_line = vec_from_first - proj
    dist_to_line = np.linalg.norm(vec_to_line, axis=1)

    elbow_index = np.argmax(dist_to_line)
    eps_opt = float(k_distances[elbow_index])

    return eps_opt

In [ ]:
def run_dbscan_with_metrics(X, eps, min_samples, min_cluster_points=10):
    if eps is None:
        return None, 0, 1.0, None, None

    model = DBSCAN(eps=eps, min_samples=min_samples)
    labels = model.fit_predict(X)

    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    noise_rate = float(np.mean(labels == -1))

    mask = labels != -1
    if len(np.unique(labels[mask])) > 1 and np.sum(mask) >= min_cluster_points:
        sil = float(silhouette_score(X[mask], labels[mask]))
        dbi = float(davies_bouldin_score(X[mask], labels[mask]))
    else:
        sil, dbi = None, None

    return labels, n_clusters, noise_rate, sil, dbi




In [ ]:
def grid_pca_dbscan(
    X_input,
    pca_dims,
    min_samples_values,
    min_cluster_points=10,
    random_state=42,
    verbose=True
):
    results = []
    labels_dict = {}
    best_row = None

    for dim in pca_dims:
        pca = PCA(n_components=dim, random_state=random_state)
        X_pca = pca.fit_transform(X_input)
        explained_var = float(pca.explained_variance_ratio_.sum())

        # lọc min_samples theo điều kiện >= dim + 1
        ms_list = [ms for ms in min_samples_values if ms >= dim + 1]

        if verbose:
            print(f"\nPCA dim={dim} | ExplainedVar={explained_var:.3f} | min_samples >= {dim+1}")
            print("-" * 70)

        for ms in ms_list:
            eps_elbow = compute_eps_elbow(X_pca, ms)

            # refine eps quanh elbow
            best_eps, best_eps_row, _ = select_best_eps_around_elbow(
                X_pca,
                min_samples=ms,
                eps_elbow=eps_elbow,
                min_cluster_points=min_cluster_points
            )

            labels, n_clusters, noise_rate, sil, dbi = run_dbscan_with_metrics(
                X_pca, best_eps, ms, min_cluster_points=min_cluster_points
            )

            labels_dict[(dim, ms)] = labels

            row = {
                "pca_dim": dim,
                "explained_variance": explained_var,
                "min_samples": ms,
                "eps_elbow": eps_elbow,
                "eps_optimal": best_eps,
                "n_clusters": n_clusters,
                "noise_rate": float(noise_rate),
                "silhouette": None if sil is None else float(sil),
                "dbi": None if dbi is None else float(dbi),
            }
            results.append(row)

            # không in dòng có sil/dbi=None
            if verbose and (sil is not None) and (dbi is not None):
                print(
                    f"✓ ms={ms:>2} | eps_elbow={eps_elbow:.5f} | eps*={best_eps:.5f} | "
                    f"clusters={n_clusters:>2} | noise={noise_rate:.1%} | Sil={sil:.3f} | DBI={dbi:.3f}"
                )

            # update best (chỉ xét cấu hình hợp lệ)
            if (sil is not None) and (dbi is not None) and (n_clusters >= 2):
                if best_row is None:
                    best_row = row
                else:
                    if (
                        (row["silhouette"] > best_row["silhouette"]) or
                        (row["silhouette"] == best_row["silhouette"] and row["dbi"] < best_row["dbi"]) or
                        (row["silhouette"] == best_row["silhouette"] and row["dbi"] == best_row["dbi"]
                         and row["noise_rate"] < best_row["noise_rate"])
                    ):
                        best_row = row

    if verbose:
        print("\nBEST CONFIG (filtered valid metrics)")
        print("=" * 70)
        if best_row is None:
            print("Không có cấu hình nào đủ điều kiện để tính Silhouette/DBI (tất cả đều None).")
        else:
            print(
                f"PCA dim={best_row['pca_dim']} | ExplainedVar={best_row['explained_variance']:.3f} | "
                f"min_samples={best_row['min_samples']} | eps_elbow={best_row['eps_elbow']:.5f} | "
                f"eps*={best_row['eps_optimal']:.5f} | clusters={best_row['n_clusters']} | "
                f"noise={best_row['noise_rate']:.1%} | Sil={best_row['silhouette']:.3f} | DBI={best_row['dbi']:.3f}"
            )

    return results, labels_dict, best_row


In [ ]:
pca_dims = [2, 3, 4, 5, 6, 8, 10]
min_samples_values = [3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]

dbscan_results, dbscan_labels, best_dbscan = grid_pca_dbscan(
    X_input=X,
    pca_dims=pca_dims,
    min_samples_values=min_samples_values,
    min_cluster_points=10,
    verbose=True
)




PCA dim=2 | ExplainedVar=0.664 | min_samples >= 3
----------------------------------------------------------------------
✓ ms= 3 | eps_elbow=0.31519 | eps*=0.37823 | clusters=20 | noise=3.4% | Sil=-0.084 | DBI=0.532
✓ ms= 4 | eps_elbow=0.35938 | eps*=0.43126 | clusters=10 | noise=3.2% | Sil=0.183 | DBI=0.531
✓ ms= 5 | eps_elbow=0.42093 | eps*=0.50512 | clusters= 6 | noise=3.1% | Sil=0.268 | DBI=0.453
✓ ms= 6 | eps_elbow=0.43972 | eps*=0.52766 | clusters= 4 | noise=3.4% | Sil=0.326 | DBI=0.445
✓ ms= 7 | eps_elbow=0.47931 | eps*=0.57517 | clusters= 3 | noise=3.4% | Sil=0.393 | DBI=0.443
✓ ms= 8 | eps_elbow=0.50227 | eps*=0.57761 | clusters= 2 | noise=4.1% | Sil=0.374 | DBI=0.476
✓ ms= 9 | eps_elbow=0.52379 | eps*=0.60235 | clusters= 2 | noise=3.9% | Sil=0.379 | DBI=0.462
✓ ms=10 | eps_elbow=0.61894 | eps*=0.58799 | clusters= 2 | noise=4.6% | Sil=0.339 | DBI=0.542
✓ ms=11 | eps_elbow=0.59351 | eps*=0.62319 | clusters= 2 | noise=4.3% | Sil=0.338 | DBI=0.553
✓ ms=12 | eps_elbow=0.69957 | e

| Silhouette      | Ý nghĩa                                             |
| --------------- | --------------------------------------------------- |
| **0.50 – 1.0**  | Các cụm rất rõ ràng, tách biệt tốt                  |
| **0.25 – 0.50** | Các cụm ổn, khá rõ                                  |
| **0.10 – 0.25** | Các cụm yếu → hơi bị lẫn nhau                       |
| **0 – 0.10**    | Các cụm không rõ ràng                               |
| **< 0**         | phân cụm sai → điểm nằm gần cụm khác hơn cụm của nó |

| DBI           | Ý nghĩa                      |
| ------------- | ---------------------------- |
| **0.0 – 0.6** | cụm rất rõ ràng              |
| **0.6 – 1.0** | cụm tốt                      |
| **1.0 – 1.5** | cụm yếu, lẫn nhau            |
| **> 1.5**     | phân cụm tệ                  |
| **> 2.0**     | gần như không có cụm thực sự |